# Finding Similar Movies

For study purpose we use pretty old dataset to create Recommendation system

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#load rating & movies info
r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3), encoding="ISO-8859-1")

m_cols = ['movie_id', 'title']
movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, usecols=range(2), encoding="ISO-8859-1")

#merge both tables
ratings = pd.merge(movies, ratings)


In [2]:
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [3]:
#construct a user / movie rating matrix

movieRatings = ratings.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRatings.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's extract a Series of users who rated Star Wars:

In [4]:
starWarsRatings = movieRatings['Star Wars (1977)']
starWarsRatings.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

Compute the pairwise correlation of Star Wars' vector of user rating with every other movie.

Drop any results that have no data, and construct a new DataFrame of movies and their correlation score (similarity) to Star Wars:

In [5]:
similarMovies = movieRatings.corrwith(starWarsRatings)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

,0
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398
2 Days in the Valley (1996),0.066654
"20,000 Leagues Under the Sea (1954)",0.289768
2001: A Space Odyssey (1968),0.230884
"39 Steps, The (1935)",0.106453


Result seems to be odd. Let's try to clean data

In [6]:
#Sort movies by correlation
similarMovies.sort_values(ascending=False)

title
Hollow Reed (1996)                                                                   1.000000
Stripes (1981)                                                                       1.000000
Full Speed (1996)                                                                    1.000000
Golden Earrings (1947)                                                               1.000000
Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)    1.000000
Star Wars (1977)                                                                     1.000000
Ed's Next Move (1996)                                                                1.000000
Scarlet Letter, The (1926)                                                           1.000000
Hurricane Streets (1998)                                                             1.000000
Safe Passage (1994)                                                                  1.000000
Outlaw, The (1943)                                    

Let's construct a new DataFrame that counts up how many ratings exist for each movie, and also the average rating.

In [7]:
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
movieStats.head(10)

rating          
                                              size      mean
title                                                       
'Til There Was You (1997)                        9  2.333333
1-900 (1994)                                     5  2.600000
101 Dalmatians (1996)                          109  2.908257
12 Angry Men (1957)                            125  4.344000
187 (1997)                                      41  3.024390
2 Days in the Valley (1996)                     93  3.225806
20,000 Leagues Under the Sea (1954)             72  3.500000
2001: A Space Odyssey (1968)                   259  3.969112
3 Ninjas: High Noon At Mega Mountain (1998)      5  1.000000
39 Steps, The (1935)                            59  4.050847

Let's get rid of any movies rated by fewer than 100 people, and check the top-rated ones that are left:

In [9]:
popularMovies = movieStats['rating']['size'] >= 150
movieStats[popularMovies].sort_values([('rating', 'mean')], ascending=False)[:15]

rating          
                                                     size      mean
title                                                              
Schindler's List (1993)                               298  4.466443
Casablanca (1942)                                     243  4.456790
Shawshank Redemption, The (1994)                      283  4.445230
Rear Window (1954)                                    209  4.387560
Usual Suspects, The (1995)                            267  4.385768
Star Wars (1977)                                      584  4.359589
Citizen Kane (1941)                                   198  4.292929
To Kill a Mockingbird (1962)                          219  4.292237
One Flew Over the Cuckoo's Nest (1975)                264  4.291667
Silence of the Lambs, The (1991)                      390  4.289744
North by Northwest (1959)                             179  4.284916
Godfather, The (1972)                                 413  4.283293
Secrets & Lies (1996)                                 162  4.265432
Good Will Hunting (1997)                              198  4.262626
Dr. Strangelove or: How I Learned to Stop Worry...    194  4.252577

Let's join this data with our original set of similar movies to Star Wars:

In [10]:
df = movieStats[popularMovies].join(pd.DataFrame(similarMovies, columns=['similarity']))

In [11]:
df.head()

,"(rating, size)","(rating, mean)",similarity
title,,,
2001: A Space Odyssey (1968),259,3.969112,0.230884
"Abyss, The (1989)",151,3.589404,0.203709
"African Queen, The (1951)",152,4.184211,0.230540
Air Force One (1997),431,3.631090,0.113164
Aladdin (1992),219,3.812785,0.191621


In [20]:
#sort them

df.sort_values(['similarity'], ascending=False)[1:10]

,"(rating, size)","(rating, mean)",similarity
title,,,
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
"Sting, The (1973)",241,4.058091,0.367538
Indiana Jones and the Last Crusade (1989),331,3.930514,0.350107
L.A. Confidential (1997),297,4.161616,0.319065
"Bridge on the River Kwai, The (1957)",165,4.175758,0.316580
E.T. the Extra-Terrestrial (1982),300,3.833333,0.303619
Batman (1989),201,3.427861,0.289344


We get recommendation list to 'Star Wars (1977)'.